<a href="https://colab.research.google.com/github/Renator12/Structured-products/blob/main/structured_products_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from statistics import NormalDist
from scipy.integrate import quad
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import random


INPUT VARIABLES

In [ ]:
# Global Variables
S0 =770.21# Initial Stock Price
K = 103
r = 0.02# interest rate
sigma = 0.1# Volatility
years =11
time_periods = 1# per year
branch_count = 10# Only use even numbers
dt = 1/time_periods
principal=100
booster_return=0.4
boost_level=0
buffer=-0.3
participation_rate=0.45
barrier=0.75
coupon_rate=0.015
coupon_threshold=0
autocall=1
payment_threshold=coupon_threshold
fixed=0.1
variable=0.1
excess_ratio_variable=0.05

In [ ]:
sigma=0.05

In [ ]:
q_value = []
phi = 0.6 # A value from 0 to 1
if branch_count == 2:
    q_value.append((1-0.5)**phi/branch_count)
elif branch_count % 2 == 0:
    for i in range(1,int(branch_count/2)+1):
        q_value.append((i-0.5)**phi/branch_count)

elif branch_count % 2 != 0:
    for i in range(1,int((branch_count+1)/2)):
        q_value.append((i-0.5)**phi/branch_count)
q_sum = sum(q_value)*2
q_transform = [x / q_sum for x in q_value]
q_full = q_transform + q_transform[::-1]
# q vector (coefficients for the zi terms)
qi = np.array(q_full)

In [ ]:
big_z = [-np.inf]

for i in range(1,len(q_transform)+1):
    big_z.append(NormalDist().inv_cdf(sum(q_transform[0:i])))
big_z = big_z + [x * -1 for x in big_z[len(big_z)-2::-1]]
# Zi vector (bounds for the zi variables)
Z_lower = np.array(big_z[0:-1])
Z_upper = np.array(big_z[1:])
# The objective function we want to minimize
def objective_function(z):
    return (np.sum(qi * z**4) - 3)**2
# The equality constraints
def constraint1(z):
    return np.sum(qi * z)
def constraint2(z):
    return np.sum(qi * z**2) - 1
# Define the constraints dictionary
con1 = {'type': 'eq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
constraints = [con1, con2]
# Define the bounds for each zi
bounds = [((None,high) if low == -np.inf else ((low,None) if high == np.inf else (low, high)) ) for low, high in zip(Z_lower, Z_upper)]
# # Define the bounds for each zi
# bounds = [((None,high) if low == -np.inf else (low, high)) for low, high in zip(Z_lower, Z_upper)]
# Initial guess (required by the minimizer, let's start with the midpoint of the bounds)
z_initial = (Z_lower + Z_upper) / 2
z_initial[0] = NormalDist().inv_cdf(sum(q_full[0:1])/2)
z_initial[-1] = z_initial[0]*-1

In [ ]:
# Run the minimizer
result = minimize(objective_function, z_initial,method='SLSQP', bounds=bounds, constraints=constraints)
# The result of minimization will be in result.x
result.x, result.success, result.message
z_value = result.x

In [ ]:
qi

array([0.03996484, 0.0772593 , 0.10496875, 0.12845081, 0.1493563 ,
       0.1493563 , 0.12845081, 0.10496875, 0.0772593 , 0.03996484])

# **BROWNIAN MOTION**

In [ ]:
def generate_btree(time_periods,years,branch_count,z_value):
    b_tree = np.zeros((time_periods*years+1,branch_count))
    for i in range (1,time_periods*years+1):
        for j in range (0,branch_count):
            b_tree[i][j] = np.sqrt(i*dt) * z_value[j]
    return b_tree

In [ ]:
def probcali(p, y, mu, sigma):
    m = len(p)
    a = np.dot(p, y) - mu
    b = np.dot(p, y**2) - mu**2 - sigma**2
    pind = np.argsort(p)  # Sorts p and returns the indices that would sort p
    x = np.zeros(m)
    wm = pind[-1]  # Index of max probability
    w2 = pind[-2]
    w1 = pind[-3]
    y1 = y[w1]  # Min
    y2 = y[w2]  # Middle
    ym = y[wm]  # Max probability
    # Calculate calibration factors
    x[wm] = (-b + a * (y1 + y2)) / (ym**2 - y2**2 - (y1 + y2) * (ym - y2))
    x[w1] = (-a - x[wm] * (ym - y2)) / (y1 - y2)
    x[w2] = -(x[wm] + x[w1])
    pc = p + x  # Adjust original probabilities
    pc = np.maximum(pc, 0)  # Ensure no negative probabilities
    pc /= np.sum(pc)  # Normalize to ensure the sum of probabilities is 1
    return pc

In [ ]:
def generate_transition_matrices(time_periods,years,branch_count,b_tree):
    transition_matrices = []
    # n-1 number of matrices
    for n in range(1, time_periods*years):
        # intialize new matrix
        transition_matrix = np.zeros((branch_count, branch_count))
        test = np.zeros((branch_count, branch_count))
        # ith columns and jth rows
        for i in range(0, branch_count):
            # j goes from 0 to number of branches
            for j in range(0, branch_count):
                if j == 0:
                    w3 = np.NINF
                else:
                    w3 = b_tree[n+1][j-1]
                if j == (branch_count-1):
                    w1 = np.inf
                else:
                    w1 = b_tree[n+1][j+1]
                w2 = b_tree[n+1][j]
                upper_bound = (w1 + w2)/2
                lower_bound = (w2 + w3)/2
                w = b_tree[n][i]
                # Check built in cdf
                f = lambda x: 1/np.sqrt(2*np.pi*dt)*np.exp(-(x-w)**2/(2*dt))
                # Use the standard normal pdf to integrate from lower to upper bound
                transition_matrix[i][j], _ = quad(f, lower_bound, upper_bound)
        for i in range(branch_count):
            mu = b_tree[n][i]
            sigma = np.sqrt(dt)
            y = b_tree[n+1]  # Assuming y values for calibration are the next step's node values
            # Calibrate probabilities
            transition_matrix[i, :] = probcali(transition_matrix[i, :], y, mu, sigma)
            # Ensure probabilities sum to 1 after calibration
            transition_matrix[i, :] /= np.sum(transition_matrix[i, :])
        transition_matrices.append(transition_matrix)
        # Append the calculated matrix to the list of transition matrices
        transition_matrices.append(transition_matrix)
    return transition_matrices

In [ ]:
def generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value):
    stock_price_tree = np.zeros((time_periods*years+1, branch_count))
    # Populate the stock price tree
    for i in range(1, time_periods*years+1):
        for j in range(branch_count):
            # Transform the Brownian motion value into a stock price
            W_t = np.sqrt(i * dt) * z_value[j]  # Brownian motion at time t
            stock_price_tree[i][j] = S0 * np.exp((r - 0.5 * sigma**2) * i * dt + sigma * W_t)
    # Set the initial stock prices
    stock_price_tree[0] = S0
    return stock_price_tree

# GENERATING SOME PRICES

In [ ]:
b_tree = generate_btree(time_periods,years,branch_count,z_value)
transition_matrices = generate_transition_matrices(time_periods,years,branch_count,b_tree)

In [ ]:
np.array(transition_matrices).shape

(22, 10, 10)

In [ ]:
prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)
prices

array([[249.65      , 249.65      , 249.65      , 249.65      ,
        249.65      , 249.65      , 249.65      , 249.65      ,
        249.65      , 249.65      ],
       [226.53903639, 237.97039073, 242.62227047, 246.71951268,
        251.8341604 , 256.94167168, 262.26823071, 266.69724037,
        271.91067605, 285.63151555],
       [220.00779208, 235.87057817, 242.41757914, 248.22724844,
        255.53577543, 262.89572179, 270.63613048, 277.12207221,
        284.81407964, 305.34945492],
       [216.06471304, 235.29617898, 243.31983066, 250.48079153,
        259.54279498, 268.72761383, 278.44975814, 286.64460053,
        296.41924344, 322.80288402],
       [213.42268061, 235.50510195, 244.80247133, 253.14040453,
        263.7447073 , 274.55133725, 286.05256449, 295.7954685 ,
        307.47300477, 339.28661053],
       [211.59330705, 236.21571107, 246.66586609, 256.07761195,
        268.10044185, 280.41150626, 293.57681031, 304.77848223,
        318.26184432, 355.29688976],
       [21

# **CUSTOM PRINCPAL PROTECTION NOTES**

1) COUPON DATES WITH VARYING FIXED RETURNS ->Notes present  a minimum fixed return after the first 2 years and then they become 0 .They also have a maximum return which can be achieved on each of the dates
https://notes.tdsecurities.com/detail/1610

In [ ]:
def payoff_coupon_ppn(S,S0,min_fixed_return ,max_return):
  index_return=(S-S0)/S0
  return np.where(index_return > 0, np.minimum(index_return, max_return) * S0, min_fixed_return * S0)

import numpy as np

def price_coupon_ppn(stock_price_tree, transition_matrices, r, min_fixed_return, max_return, years, S0, qi):
    dt = 1
    # Initialize option value tree (same shape as stock_price_tree)
    option_value_tree = np.zeros_like(stock_price_tree)

    # Calculate payoffs at maturity (final period)
    prices = stock_price_tree[-1]  # Stock prices at the last time step
    option_value_tree[-1] = payoff_coupon_ppn(prices, S0, min_fixed_return, max_return)+S0 # Payoffs at maturity

    # Backward induction for each time step
    for t in range(years - 1, 0, -1):
        prices_at_t = stock_price_tree[t]  # Stock prices at time t
        if t <= 2:
            payoff = payoff_coupon_ppn(prices_at_t, S0, min_fixed_return, max_return)
        else:
            payoff = payoff_coupon_ppn(prices_at_t, S0, 0, max_return)

        # Calculate the discounted expected value at each node using the transition matrix
        dot_prod = np.dot(transition_matrices[t - 1], option_value_tree[t+1]) * np.exp(-r * dt)

        # Add the payoff for this time step
        option_value_tree[t] = dot_prod + payoff

    # Calculate the value at the root node (t = 0)
    option_value_tree[0] = np.dot(qi, option_value_tree[1]) * np.exp(-r * dt)

    return option_value_tree[0][0] # Return the value of the structured note at t = 0


In [ ]:
payoff_coupon_ppn(prices[-1],100,0.0325,0.06)

array([6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])

In [ ]:
np.array(transition_matrices).shape

(22, 10, 10)

In [ ]:
price_coupon_ppn(prices,transition_matrices,r,0.0325,0.06,years,400,qi)

352.5518954255628

# **PRINCIPAL AT RISK NOTES**

1)   **PAYOFF FOR A PRINCIPAL AT RISK NOTE WITH VARYING FIXED RETURNS , AUTOCALL FOR EARLY PREMIUM AND BARRIERS FOR PARTIAL DOWNSIDE PROTECTION**
def payoff_changing_fixed_rate->Non-vectorized payoff function
    

In [ ]:
def payoffs_changing_fixed_rate(prinicpal , prices,S0 , fixed_rate,excess_return_variable,auto_call,barrier):#autocall 105% ->input 1.05
  returns = (prices-S0)/S0
  count=0
  for i,price in zip(returns,prices):
    #print(i)
    if price>S0*auto_call:
      excess_ratio=max(0,(i-fixed_rate)*excess_return_variable)
      #print(excess_ratio)
      returns[count]=principal*(1+fixed_rate+excess_ratio)
    elif price>S0*barrier:
      returns[count]=principal
    else :
      returns[count]=principal*(1+i)
    count+=1
  return returns
def price_auto_call_changing_fixed_rate(principal,stock_price_tree, transition_matrices, r, fixed, variable, years,excess_ratio_variable ,barrier,auto_call=1):
  '''
  price_auto_call2 : Uses a willow tree method to price a structured product with varying fixed rates per year , increasing at a constant rate and having an autocall value.Payoff is given either on autocall day where
  Payoff=S0*(1+Fixed_rate + excess_ratio) .
  '''
  year_count = years - 1 #since we already calculate the final year seperately
  Final_rate = fixed #+ (year_count) *variable  # Calculate the initial Final Rate
  # Initialize option value tree
  option_value_tree = np.zeros_like(stock_price_tree)
  prices=stock_price_tree[-1]
  # Calculate payoffs at maturity (final period)
  option_value_tree[-1] = payoffs_changing_fixed_rate(principal , prices,S0 , fixed,excess_ratio_variable,auto_call,barrier) #final layer
  # Backward induction for each time step
  for t in range(time_periods * years - 1, 0, -1):
      prices=stock_price_tree[t]
      payoff = payoffs_changing_fixed_rate(principal,stock_price_tree[t],S0,Final_rate,excess_ratio_variable,auto_call,barrier)
      dot_prod = np.dot(transition_matrices[t - 1], option_value_tree[t + 1]) * np.exp(-r * dt)
      option_value_tree[t] = np.where(prices >=S0*auto_call, payoff,dot_prod)
      # Reduce the Final_rate at every auto-callable date (each period)
      Final_rate += variable  # Decrease Final_rate by the specified decrement
  option_value_tree[0] = np.dot(qi, option_value_tree[1]) * np.exp(-r * dt)
  return option_value_tree[0][0]

In [ ]:
price_auto_call_changing_fixed_rate(principal,prices, transition_matrices, r, fixed, variable, years,excess_ratio_variable ,barrier,auto_call=1)

nan

In [ ]:
#Example of using this payoff function
pay1=payoffs_changing_fixed_rate(principal , prices[-1],S0 , fixed,excess_ratio_variable,autocall,barrier)
print('prices at prices[-1] are ',prices[-1],'\n', 'payoff is ',pay1)

prices at prices[-1] are  [209.26386832 248.17401573 265.3881762  281.23891437 301.95659065
 323.70629905 347.55236558 368.31049119 393.85770665 467.09086222] 
 payoff is  [100. 100. 100.  nan  nan  nan  nan  nan  nan  nan]


# 2) Payoff for simple path independent notes which have either a buffer or a barrier . The barrier part of the code is incomplete since I did not encounter any path independent notes with only a barrier in the principal at risk section

def path_independent_buffer_barrier():
 Vectorized function for the path independent buffer principal at risk. Since the only payoff needed is at the final time step ,this is a very simple implementation using numpy.

In [ ]:
#buffer protections
def path_independent_buffer_barrier(principal, prices, S0, participation_rate, buff=True, barrier=1, buffer=1,buffer_variable=0.3):
    returns = (prices - S0) / S0
    #print(returns)
    if buff:   #always going to be using buff until we add the barrier code if needed !
        # Calculate the payoff using element-wise conditions
        ans = np.where(
            returns > 0,
            principal * (1 + participation_rate * returns),  # When returns are positive
            np.where(
                (returns < 0) & (returns >= buffer),  # Returns are negative but within the buffer
                principal,  # Principal is protected within the buffer
                principal * (1 + returns + buffer_variable)  # When returns are below the buffer
            )
        )
    #add the barrier partt later
    return ans
def price_buffer_barrier(stock_price_tree,transition_matrices,r,years,participation_rate,buffer):

    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)

    # Payoff function takes in the stock prices and a fixed rate to get a payoff vector
    option_value_tree[-1] = path_independent_buffer_barrier(principal, prices[-1], S0, participation_rate, buff=True, barrier=1, buffer=buffer)
    # Backward induction to calculate option price at each node
    for t in range(years-1, 0, -1):
        option_value_tree[t] = np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r*dt)

    option_value_tree[0] = np.dot(qi,option_value_tree[1])*np.exp(-r*dt)

    return option_value_tree[0][0]

In [ ]:
price_buffer_barrier(prices,transition_matrices,r,years,participation_rate,buffer)

103.19965590970853

In [ ]:
path_independent_buffer_barrier(principal , prices[-1],S0 , participation_rate,buff=True,barrier=1,buffer=0.3)

array([ 84.67602111, 105.79053074, 116.17585056, 126.3004053 ,
       104.65356734, 111.72725417, 119.99823827, 127.63469514,
       137.58758012, 169.48081667])

#3) PATH DEPENDENT COUPON NOTES WITH BARRIERS.Payoff Functions depend on if the day is the final exercise date or not .
def barrier_coupon_participation():->
vectorized function dealing with notes which have no coupon/payment thresholds and provide coupons regularly. Payoff for the final day checks
a)If Index Return >0:
     get principal *(1+(participation_rate*index_return))
b)If Index Return <0 and price >barrier
     get principal
c)if Index Return < barrier:
     get principal * (1+index_return)



In [ ]:
#BARRIER COUPONS STANDARD WITH NO AUTOCALL and with coupon threshold/no coupon thresholds
#https://notes.tdsecurities.com/detail/1995https://notes.tdsecurities.com/detail/1995
#https://notes.tdsecurities.com/detail/1996https://notes.tdsecurities.com/detail/1996
def barrier_coupon_participation(principal, prices, S0, barrier_level , coupon_rate,participation_rate,is_final_day=False,threshold=-1000):#always pays coupons without threshold
  results=[]
  index_return=(prices-S0)/S0
  if is_final_day:
    results=np.where(index_return>0,principal*(1+(participation_rate*index_return)),np.where((index_return <=0) & (prices>=S0*barrier_level),principal,principal*(1+index_return)))
  else:
    results=np.where(index_return>=threshold,principal*coupon_rate,0) #since threshold is so low , basically free
  return results

def barrier_coupon(principal, prices, S0, barrier_level , coupon_rate,is_final_day=False,coupon_threshold=-1000):
  results=[]
  index_return=(prices-S0)/S0
  for price,ret in zip(prices,index_return):
      if is_final_day:
        if ret>=coupon_threshold:
          if price>=S0*barrier_level:
            results.append(principal+(coupon_rate*principal)) #principal +coupon
          else:
            results.append((principal*(1+ret)+(coupon_rate*principal))) #depreciated principal added to coupon
        else:
          results.append(0)
      else:
          if ret>=coupon_threshold: #not final day
            results.append(coupon_rate*principal)
          else:
            results.append(0)

  return results

def barrier_coupon_pricing(principal, prices, S0, participation_rate,barrier, coupon_rate, years, transition_matrices, r, qi,threshold_or_not=False,coupon_threshold=1,fun='barrier_coupon'):
  year_count = years - 1
  option_value_tree = np.zeros_like(prices)
  final_prices=prices[-1]
  if fun=='barrier_coupon': #payoff is coupon_rate*principal
    option_value_tree[-1]=barrier_coupon(principal,final_prices,S0,barrier,coupon_rate,is_final_day=True,coupon_threshold=coupon_threshold)
  elif fun=='barrier_coupon_participation':
    option_value_tree[-1]=barrier_coupon_participation(principal,final_prices,S0,barrier,coupon_rate,participation_rate,is_final_day=True)
  #print('final day is ',option_value_tree[-1])
  for t in range(years - 1, 0, -1):
    current_price=prices[t]
    if fun=='barrier_coupon':
      payoff=barrier_coupon(principal,current_price,S0,barrier,coupon_rate,is_final_day=False,coupon_threshold=coupon_threshold)
    elif fun=='barrier_coupon_participation':
      payoff=barrier_coupon_participation(principal,current_price,S0,barrier,coupon_rate,participation_rate,is_final_day=False)
   #print('PAYOFF AT ',t,'=>',payoff)
    prod=np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r)
    #print('prod at ',t,'is',prod)
    option_value_tree[t]=prod+payoff

  option_value_tree[0]=np.dot(qi,option_value_tree[1])*np.exp(-r)
  return option_value_tree[0][0]


In [ ]:
barrier_coupon_pricing(principal, prices, S0,participation_rate, barrier, coupon_rate, years, transition_matrices, r, qi,coupon_threshold=coupon_threshold,fun='barrier_coupon')

62.39651273304606

4)Complicated path dependent function which includes possible early activation due to autocall , coupons subject to payment thresholds and barrier on final day.


In [ ]:

def barrier_autocall_coupon(principal, prices, S0, barrier_level, autocall_level, coupon_rate, payment_threshold, is_final_day=False):
    """
    Combined function for barrier, autocall, and coupon payout calculations.
    :param principal: Initial investment principal
    :param prices: List of asset prices at different times
    :param S0: Initial price of the asset
    :param barrier_level: Level of the barrier for the barrier condition
    :param autocall_level: Level of autocall trigger
    :param coupon_rate: The coupon rate to apply for payouts
    :param payment_threshold: Minimum return for coupon payment eligibility
    :param is_final_day: Boolean to check if it's the final day for the barrier conditions
    :return: List of payoffs based on the given conditions
    """
    results = np.zeros_like(prices)  # Initialize the results array with the same shape as prices
    index_return = (prices - S0) / S0  # Calculate index return for each price
    count = 0
    for price, ret in zip(prices, index_return):
        if price >= S0 * autocall_level:  # Autocall condition
            results[count] = principal * (1 + coupon_rate)

        elif S0 * barrier_level <= price < S0 * autocall_level:  # Barrier condition
            if is_final_day:  # If it's the final day
                if ret >= payment_threshold:
                    results[count] = principal * (1 + coupon_rate)
                else:
                    results[count] = principal
            else:  # Non-final day (only consider partial coupon payouts)
                if ret >= payment_threshold:
                    results[count] = principal * coupon_rate
                else:
                    results[count] = 0

        else:  # Below the barrier level
            if is_final_day:
                results[count] = principal * (1 + ret)  # Final day, return based on asset performance
            else:
                results[count] = 0  # No payment on non-final day below barrier
        count += 1
    return results
def payoff_autocall_coupon(principal, prices, S0, barrier, autocall, coupon_rate, payment_threshold, years, transition_matrices, r, qi):
    dt = 1
    option_value_tree = np.zeros_like(prices)
    option_value_tree[-1] = barrier_autocall_coupon(principal, prices[-1], S0, barrier, autocall, coupon_rate, payment_threshold,is_final_day=True)
    for t in range(years - 1, 0, -1):
        # Make sure prices is indexed properly to access historical prices
        payoff = barrier_autocall_coupon(principal, prices[t], S0, barrier, autocall, coupon_rate, payment_threshold, is_final_day=False)
        # Make sure option_value_tree[t + 1] is indexed properly
        prod = np.dot(transition_matrices[t - 1], option_value_tree[t + 1]) * np.exp(-r * dt)
        # Check the condition and calculate the option value
        option_value_tree[t] = np.where(payoff == principal * (1 + coupon_rate), payoff,
                                          np.where(payoff == principal * coupon_rate, payoff + prod, prod))
    option_value_tree[0] = np.dot(qi, option_value_tree[1]) * np.exp(-r * dt)
    return option_value_tree[0][0]

In [ ]:
prices

array([[ 770.21      ,  770.21      ,  770.21      ,  770.21      ,
         770.21      ,  770.21      ,  770.21      ,  770.21      ,
         770.21      ,  770.21      ],
       [ 620.09821906,  684.25855155,  711.2719982 ,  735.4978092 ,
         766.30854237,  797.70713585,  831.12387724,  859.43182187,
         893.36082803,  985.79505394],
       [ 571.84573421,  657.27982762,  694.27412591,  727.9501954 ,
         771.44715516,  816.52558588,  865.31515748,  907.28764345,
         958.35335981, 1101.53184916],
       [ 539.26076688,  639.52994426,  683.88983042,  724.73625696,
         778.12449875,  834.17222702,  895.62214778,  949.11462514,
        1014.9483073 , 1203.66587213],
       [ 514.44701255,  626.41211561,  676.84792898,  723.73978719,
         785.64620167,  851.34705009,  924.1685817 ,  988.19474871,
        1067.75961977, 1300.14847565],
       [ 494.41511083,  616.17694216,  671.90209296,  724.15428569,
         793.74854546,  868.31947877,  951.76861174, 1025

In [ ]:
payoff_autocall_coupon(principal, prices, S0, barrier, autocall, coupon_rate, payment_threshold, years, transition_matrices, r, qi)

95.80167178232749

# **BOOSTER**
# 5) Path independent notes with boosted returns possibility based on index returns

In [ ]:

def payoff_booster(principal, prices, S0, booster_return, boost_level, participation_rate,barrier):
    returns = np.zeros_like(prices)  # Initialize an array for returns
    count = 0
    for price in prices:
        return_i = (price - S0) / S0  # Calculate return based on current price
        net_index_return = return_i - booster_return
        if return_i >= booster_return:
            returns[count] = principal * (1 + booster_return + (participation_rate * net_index_return))
        elif boost_level <= return_i <= booster_return:
            returns[count] = principal * (1 + booster_return)
        elif return_i>=barrier and return_i<boost_level:
            returns[count]=principal
        else:
            returns[count] = principal * (1 + return_i)
        count += 1
    return returns

def price_booster_PAR(principal, stock_price_tree, transition_matrices, years, qi, S0, booster_return, boost_level, participation_rate, r,barrier):
    dt=1
    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)
    # Payoff function takes in the stock prices and returns payoff vector
    option_value_tree[-1] = payoff_booster(principal, stock_price_tree[-1], S0, booster_return, boost_level, participation_rate,barrier)
    #print(option_value_tree)
    # Backward induction to calculate option price at each node
    for t in range(years-1 , 0, -1):
        option_value_tree[t] = np.dot(transition_matrices[t-1], option_value_tree[t+1]) * np.exp(-r * dt)
    # Calculate initial option price at t=0
    option_value_tree[0] = np.dot(qi, option_value_tree[1]) * np.exp(-r * dt)
    return option_value_tree[0][0]

In [ ]:
len(prices)

12

In [ ]:
price_booster_PAR(principal,prices,transition_matrices,years,qi,S0,booster_return,boost_level,participation_rate,r,barrier)

100.81607295590844

# **IMPLIED VOLATILITY MINIMIZATION**

In [ ]:
from scipy.optimize import brentq,newton
# Define a function to find the implied volatility
def find_implied_volatility(sigma,market_price,years,model):
    def difference(sigma):
        prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)
       # print('creating prices with S0 is ',S0,'years is ',years,'barrier is ',barrier)
        print('SIGMA IS ',sigma)
        # Parameters for auto call function
        # price_auto_call(stock_price_tree,transition_matrices,r,fixed,variable,years,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0):
        match model:
          case "auto_call_coupon":
            model_price=payoff_autocall_coupon(principal, prices, S0, barrier, autocall, coupon_rate, payment_threshold, years, transition_matrices, r, qi)
          case 'buffer_or_barrier':
            model_price=price_buffer_barrier(prices,transition_matrices,r,years,participation_rate,buffer)
          case "booster_par":
            model_price= price_booster_PAR(principal, prices, transition_matrices, years, qi, S0, booster_return, boost_level, participation_rate, r,barrier)
          case "booster_par_with_barrier":
            model_price=price_booster_PAR(principal, prices, transition_matrices, years, qi, S0, booster_return, boost_level, participation_rate, r,barrier)
          case "barrier_coupon":
            model_price=barrier_coupon_pricing(principal, prices, S0, barrier,participation_rate, coupon_rate, years, transition_matrices, r, qi,threshold_or_not=False,coupon_threshold=coupon_threshold)

          case 'barrier_coupon_participation':
             model_price=barrier_coupon_pricing(principal, prices, S0,participation_rate, barrier, coupon_rate, years, transition_matrices, r, qi,coupon_threshold=coupon_threshold,fun="barrier_coupon_participation")
          case 'autocall_changing_fixed_rate':
            model_price=price_auto_call_changing_fixed_rate(principal,prices, transition_matrices, r, fixed, variable, years,excess_ratio_variable ,barrier,auto_call=1)
        difference = (model_price - market_price)
        print('model price is ',model_price , 'market price ius ',market_price)
        return difference
    # Use the newton method to find the root of the difference function
    #sigma = newton(difference, sigma,tol=1e-6)
    sigma = brentq(difference, a=sigma, b=2, xtol=1e-6)
    #print(f"Implied volatility (sigma) found: {sigma}")
    return sigma

In [ ]:
payoff_autocall_coupon(principal, prices, S0, barrier, autocall, coupon_rate, payment_threshold, years, transition_matrices, r, qi)

97.19716108177778

In [ ]:
price_buffer_barrier(prices,transition_matrices,r,years,participation_rate,buffer)

90.71752651757659

In [ ]:
market_price = 97.25# Insert the market price here
# Find the implied volatility
implied_volatility = find_implied_volatility(sigma,market_price,years,"barrier_coupon")

SIGMA IS  0.1
model price is  62.39651273304606 market price ius  97.25
SIGMA IS  2.0
model price is  0.3828542194816767 market price ius  97.25


ValueError: f(a) and f(b) must have different signs

# USING CSV FILES TO DIRECTLY GENERATE RESULTS . ENTER I

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
ls

 sample_data/  'STRUCTURES NOTES PAR DC TD - Sheet1.csv'


In [ ]:
import pandas as pd
df=pd.read_csv('STRUCTURES NOTES PAR DC TD - Sheet1.csv')

In [ ]:
df.rename(columns={'Boost level ':'Boost level','Participation rate\t':'Participation rate','Boost level\t':'Boost level','Coupon Rate\t':'Coupon Rate'},inplace=True)

In [ ]:
df.columns

Index(['LINK(PAGE)', 'PAYOFF', 'Initial', 'TYPE', 'CUSIP', 'START', 'END',
       'DURATION', 'Periods', 'INDEX', 'Boosted return', 'fixed_rate_start',
       'fixed_rate_incr', 'Participation rate', 'Boost level', 'Barrier',
       'Coupon Rate', 'Threshold', 'Autocall', 'Buffer', 'REALIZED VOL',
       'IMPLIED VOL', 'Market price'],
      dtype='object')

In [ ]:
df_altered=df.loc[:,['TYPE','Initial','Periods','Participation rate','Boost level','Autocall','fixed_rate_start','fixed_rate_incr','Buffer','Barrier','Coupon Rate','Threshold','Market price']]

In [ ]:
df_altered

,TYPE,Initial,Periods,Participation rate,Boost level,Autocall,fixed_rate_start,fixed_rate_incr,Buffer,Barrier,Coupon Rate,Threshold,Market price
0,auto_call_coupon,249.65000,12,NaN,NaN,1.1,NaN,NaN,NaN,0.80,0.0425,0.20,100.0
1,auto_call_coupon,33.29000,6,NaN,NaN,1.1,NaN,NaN,NaN,0.60,0.0645,-0.40,100.0
2,booster_par_with_barrier,48.24000,5,3.0,0.65,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,96.0
3,autocall_changing_fixed_rate,411.13000,5,NaN,NaN,NaN,0.10,0.0500,NaN,0.80,NaN,NaN,97.5
4,auto_call_coupon,250.39000,12,NaN,NaN,1.1,NaN,NaN,NaN,0.85,0.0415,-0.15,97.5
5,booster_par,72.08295,6,2.0,-0.20,NaN,NaN,NaN,0.0,NaN,NaN,NaN,96.0
6,auto_call_coupon,909.21000,12,NaN,NaN,1.1,NaN,NaN,NaN,0.70,0.0250,-0.30,98.0
7,auto_call_coupon,3658.79000,12,NaN,NaN,1.1,NaN,NaN,NaN,0.75,0.0275,-0.25,97.5
8,autocall_changing_fixed_rate,87.83833,5,NaN,NaN,NaN,0.13,0.0375,NaN,0.80,NaN,NaN,NaN
9,auto_call_coupon,43.09000,6,NaN,NaN,1.1,NaN,NaN,NaN,0.80,0.1500,-0.20,97.5


In [ ]:
def load_csv(file_name):
    df = pd.read_csv(file_name)
    df_altered=df.loc[:,['TYPE','Initial','Periods','Participation rate','Boost level','Barrier','Coupon Rate','Threshold','Market price']]
    return df

def generate_data(S0,r,sigma,dt,branch_count,years,z_value,time_periods=1):
    b_tree = generate_btree(time_periods,years,branch_count,z_value)
    transition_matrices = generate_transition_matrices(time_periods,years,branch_count,b_tree)
    prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)
    return transition_matrices,prices




In [ ]:
df_altered.head()
df_altered['Participation rate'].fillna(1,inplace=True)
df_altered['Boost level'].fillna(-100,inplace=True)
df_altered['Barrier'].fillna(100,inplace=True)
df_altered['Threshold'].fillna(-100,inplace=True)
df_altered['Autocall'].fillna(1,inplace=True)

<ipython-input-120-6e3cc5b8cd3e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_altered['Participation rate'].fillna(1,inplace=True)
<ipython-input-120-6e3cc5b8cd3e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [ ]:
df_altered.head(10)

,TYPE,Initial,Periods,Participation rate,Boost level,Autocall,fixed_rate_start,fixed_rate_incr,Buffer,Barrier,Coupon Rate,Threshold,Market price
0,auto_call_coupon,249.65000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.80,0.0425,0.20,100.0
1,auto_call_coupon,33.29000,6,1.0,-100.00,1.1,NaN,NaN,NaN,0.60,0.0645,-0.40,100.0
2,booster_par_with_barrier,48.24000,5,3.0,0.65,1.0,NaN,NaN,NaN,-0.25,NaN,-100.00,96.0
3,autocall_changing_fixed_rate,411.13000,5,1.0,-100.00,1.0,0.10,0.0500,NaN,0.80,NaN,-100.00,97.5
4,auto_call_coupon,250.39000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.85,0.0415,-0.15,97.5
5,booster_par,72.08295,6,2.0,-0.20,1.0,NaN,NaN,0.0,100.00,NaN,-100.00,96.0
6,auto_call_coupon,909.21000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.70,0.0250,-0.30,98.0
7,auto_call_coupon,3658.79000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.75,0.0275,-0.25,97.5
8,autocall_changing_fixed_rate,87.83833,5,1.0,-100.00,1.0,0.13,0.0375,NaN,0.80,NaN,-100.00,NaN
9,auto_call_coupon,43.09000,6,1.0,-100.00,1.1,NaN,NaN,NaN,0.80,0.1500,-0.20,97.5


In [ ]:

arr = []
count = 1
for x in df_altered.iterrows():
    model, S0, years, participation_rate, boost_level, barrier, coupon_rate, coupon_threshold, market_price,autocall,fixed,variable,buffer = (
        x[1]['TYPE'], x[1]['Initial'], x[1]['Periods'], x[1]['Participation rate'],
        x[1]['Boost level'], x[1]['Barrier'], x[1]['Coupon Rate'],
        x[1]['Threshold'], x[1]['Market price'],x[1]['Autocall'],x[1]['fixed_rate_start'],x[1]['fixed_rate_incr'],x[1]['Buffer']
    )
    transition_matrices,prices=generate_data(S0,r,sigma,dt,branch_count,years,z_value,time_periods=1)

    # Print the intermediate variables
    print(f"Step {count}: model={model}, S0={S0}, years={years}, participation_rate={participation_rate}, "
          f"boost_level={boost_level}, barrier={barrier}, coupon_rate={coupon_rate}, "
          f"coupon_threshold={coupon_threshold}, market_price={market_price}")

    try:
        imp = find_implied_volatility(sigma, market_price, years, model)
    except Exception as e:
        print(f"Error encountered: {e}")
        break
        imp = np.nan

    arr.append(imp)
    count += 1

df_altered['Implied Volatility'] = arr


Step 1: model=auto_call_coupon, S0=249.65, years=12, participation_rate=1.0, boost_level=-100.0, barrier=0.8, coupon_rate=0.0425, coupon_threshold=0.2, market_price=100.0
SIGMA IS  0.1
model price is  97.19716108177778 market price ius  100.0
SIGMA IS  2.0
model price is  16.359753075092442 market price ius  100.0
Error encountered: f(a) and f(b) must have different signs


ValueError: Length of values (0) does not match length of index (13)

In [ ]:
df_altered

,TYPE,Initial,Periods,Participation rate,Boost level,Autocall,fixed_rate_start,fixed_rate_incr,Buffer,Barrier,Coupon Rate,Threshold,Market price,Implied Volatility
0,auto_call_coupon,249.65000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.80,0.0425,0.20,100.0,NaN
1,auto_call_coupon,33.29000,6,1.0,-100.00,1.1,NaN,NaN,NaN,0.60,0.0645,-0.40,100.0,0.129242
2,booster_par_with_barrier,48.24000,5,3.0,0.65,1.0,NaN,NaN,NaN,-0.25,NaN,-100.00,96.0,1.483183
3,autocall_changing_fixed_rate,411.13000,5,1.0,-100.00,1.0,0.10,0.0500,NaN,0.80,NaN,-100.00,97.5,0.387244
4,auto_call_coupon,250.39000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.85,0.0415,-0.15,97.5,NaN
5,booster_par,72.08295,6,2.0,-0.20,1.0,NaN,NaN,0.0,100.00,NaN,-100.00,96.0,1.201627
6,auto_call_coupon,909.21000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.70,0.0250,-0.30,98.0,NaN
7,auto_call_coupon,3658.79000,12,1.0,-100.00,1.1,NaN,NaN,NaN,0.75,0.0275,-0.25,97.5,NaN
8,autocall_changing_fixed_rate,87.83833,5,1.0,-100.00,1.0,0.13,0.0375,NaN,0.80,NaN,-100.00,NaN,NaN
9,auto_call_coupon,43.09000,6,1.0,-100.00,1.1,NaN,NaN,NaN,0.80,0.1500,-0.20,97.5,0.276455


In [ ]:
df_altered

In [ ]:
price_booster_PAR(principal, prices, transition_matrices, years, qi, S0, booster_return, boost_level, participation_rate, r,barrier)

In [ ]:
df_altered.groupby(['TYPE']).mean()['Implied Volatility'].sort_values(ascending=False)

In [ ]:
df_altered.to_csv('struc_notes_par_dc.csv')